<a href="https://colab.research.google.com/github/luudanmatcuoi-vn/Vietnamese-Font-Finder/blob/main/Vietnamese_Font_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/luudanmatcuoi-vn/Vietnamese-Font-Finder/refs/heads/main/Inconsolata-Bold.ttf
!wget https://raw.githubusercontent.com/luudanmatcuoi-vn/Vietnamese-Font-Finder/refs/heads/main/font_hash.json
!pip install imagehash

--2025-01-24 04:48:14--  https://raw.githubusercontent.com/luudanmatcuoi-vn/Vietnamese-Font-Finder/refs/heads/main/Inconsolata-Bold.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102152 (100K) [application/octet-stream]
Saving to: ‘Inconsolata-Bold.ttf.1’

Inconsolata-Bold.tt 100%[===================>]  99.76K  --.-KB/s    in 0.02s   

2025-01-24 04:48:14 (4.19 MB/s) - ‘Inconsolata-Bold.ttf.1’ saved [102152/102152]

--2025-01-24 04:48:14--  https://raw.githubusercontent.com/luudanmatcuoi-vn/Vietnamese-Font-Finder/refs/heads/main/font_hash.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443...

In [63]:
#@Chạy cái này, upload file và trả lời 1 câu hỏi.
import sys, re, json
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageOps
import imagehash
from scipy import ndimage
global_font = "Inconsolata-Bold.ttf"

def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

def get_image(image_path):
	with Image.open(image_path) as image:
		im_arr = np.frombuffer(image.tobytes(), dtype=np.uint8)
		im_arr = np.copy(im_arr)
		im_arr = im_arr.reshape((image.size[1], image.size[0], 3))
		im_arr = im_arr[:,:,0]
		im_arr[im_arr<100] = 0
	return np.array(im_arr, dtype=bool)

def make_seperate_img(np_img, box, label, extend = True):
	if extend:
		np_img = np.copy(np_img[box[1]:box[3]+2,box[0]:box[2]+2])
		if type(label)!=type(0):
			for g in label:
				np_img[np_img==g] = 500
			label = 500
		np_img[np_img!=label]=0
		np_img[np_img>0]=1
		return  np_img
	else:
		np_img = np.copy(np_img[box[1]:box[3],box[0]:box[2]])
		if type(label)!=type(0):
			for g in label:
				np_img[np_img==g] = 500
			label = 500
		return  np_img==label
	return  np_img==label

# image_path = "m_test_find_font.jpeg"

image_path = upload_files()[0]

image = get_image(image_path)
img = Image.open(image_path)
w,h = img.size
font = ImageFont.truetype(global_font, w//70)

# #invert
inp = input('Ảnh nền trắng chữ đen? Nhấn enter nếu đúng, điền bất cứ ký tự nào nếu sai.')
if any([gw==inp.lower() for gw in ["","dung","yes","y"] ]):
  image = (1-image)
  img = ImageOps.invert(img)

label_img, num_features = ndimage.label(image, structure = [[0,1,0],[1,1,1],[0,1,0]])
bounding_box = ndimage.find_objects(label_img)

for g in range(1,num_features+1):
  box = [ bounding_box[g-1][1].start, bounding_box[g-1][0].start, bounding_box[g-1][1].stop, bounding_box[g-1][0].stop]
  draw = ImageDraw.Draw(img)
  draw.text(( box[0],box[1] ), str(g), (255), font = font)

img

IndexError: list index out of range

In [66]:
# Điền các chữ cái với số được đánh dấu theo mẫu rồi chạy
# LƯU Ý: KHÔNG ĐIỀN CHỮ TIẾNG VIỆT
text_result = """
P 1626 1631
y 1635
t 1628
h 1625
"""

text_result = text_result.split("\n")
text_result = filter(None, text_result)
text_result = [{"text":t.split(" ")[0],"id":[int(k) for k in t.split(" ")[1:]]} for t in text_result]

for t in text_result:
	g = t["id"]
	boxs = []
	for ga in g:
		boxs += [ [ bounding_box[ga-1][1].start, bounding_box[ga-1][0].start, bounding_box[ga-1][1].stop, bounding_box[ga-1][0].stop] ]
	box = [min([t[0] for t in boxs]), min([t[1] for t in boxs]), max([t[2] for t in boxs]), max([t[3] for t in boxs])]
	temp_np_img = make_seperate_img(label_img,box,g)
	t["hash"] = imagehash.phash( Image.fromarray( temp_np_img ) )

with open('font_hash.json', 'r', encoding="utf-8") as f:
	font_data =  json.load(f)

font_result = []

for f in font_data.keys():
	score = 0
	for t in text_result:
		score+=  imagehash.hex_to_hash( font_data[f][ t["text"] ] ) - t["hash"]
	font_result+=[{"font":f,"score":score}]

font_result = sorted(font_result, key=lambda d: d['score'])
import urllib.parse
font_result = font_result[:20]
for f in font_result:
	# encoded = base64.b64encode(open(png_file_name, "rb").read())
	encode = "https://raw.githubusercontent.com/luudanmatcuoi-vn/Vietnamese-Font-Finder/main/font_images/"+ urllib.parse.quote(f["font"].split("\\")[-1]) +".jpg?raw=1"
	f["encode"] = encode
# print(font_result)

#Showing font
template = """<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<title>Font Display</title>
	<style>
body{font-family:Arial,sans-serif;max-width:1200px;margin:0 auto;padding:20px;background-color:#f5f5f5}h1{text-align:center;color:#333;margin-bottom:30px}h2{text-align:center;color:#333}.font-container{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:20px;margin-bottom:40px}.font-card{background:#fff;border-radius:8px;padding:20px;box-shadow:0 2px 4px rgba(0,0,0,.1)}.font-info{margin-bottom:10px;padding-bottom:10px;border-bottom:1px solid #eee}.font-name{font-weight:700;color:#444}.font-score{color:#666;font-size:.9em}.font-example{font-size:1.2em;margin-top:15px;line-height:1.4}
.font-img{max-width: 100%; height: auto%;}
	</style>
</head>
<body><h1>Các phông chữ tìm được</h1><h2>Tải phông chữ tại <a href="https://git.linuxholic.com/boydaihungst/vietnamese-font/">đây</a></h2><div class="font-container"><script>const fontResults = FONT_RESULT_STR;const container = document.querySelector('.font-container');fontResults.forEach(result => {const fontPath = result.font.replace('\\\\', '/');const fontName = result.font.split('\\\\').pop().replace('.ttf', '');const fontCard = document.createElement('div');fontCard.className = 'font-card';fontCard.innerHTML = `<div class="font-info"><div class="font-name">${fontName}</div><div class="font-score">Điểm: ${result.score}</div></div><div class="font-example" style="font-family: '${fontName}'"><img class="font-img" src="${result.encode}"></div>`;container.appendChild(fontCard);});</script></div></body></html>"""
template = template.replace("FONT_RESULT_STR",json.dumps(font_result).replace("\\\\","/"))
with open('show_font.html', 'w') as f:
	f.write(template)

import IPython
IPython.display.HTML('show_font.html')
